In [1]:
import os
import json
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

import mlflow
import mlflow.sklearn

In [2]:
df = pd.read_csv('final_df.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head()

,builtup_area,bedrooms,bathrooms,balcony,facing,gas_pipline,gated_community,swimming_pool,gym,intercom,power_backup,garden,sports,current_floor,total_floor,lease_type,covered_parking,open_parking,school,bus_stop,railway,mall,metro_station,hospital,restaurant,rent,latitude,longitude,furnish_encoded
0,550,1.0,2,0,West,0,1,1,1,1,1,1,1,25,34,Family,1,0,0,0,1,0,1,1,0,41000,19.1627,72.8398,1
1,954,2.0,2,0,East,0,1,1,1,1,1,1,1,12,18,Bachelor Company Family,1,0,1,0,0,0,0,1,1,70000,19.0160,72.8760,0
2,550,1.0,2,0,West,0,0,0,0,0,0,0,0,1,4,Bachelor Company Family,0,0,0,0,0,0,1,1,0,50000,19.1300,72.8250,2
3,1500,3.0,3,0,West,0,0,0,0,1,1,0,0,5,7,Family,1,0,1,0,0,0,0,1,1,240000,19.0560,72.8300,1
4,2800,3.0,3,0,none,1,1,1,1,1,1,1,1,6,56,Bachelor Company,1,1,1,0,0,0,0,1,1,350000,19.0160,72.8660,1


In [5]:
df.facing.value_counts()

facing
East          7878
none          7225
North-East    3098
West          2945
North         1517
North-West     840
South-West     467
South          404
South-East     397
Name: count, dtype: int64

In [6]:
RANDOM_STATE = 42
TEST_SIZE = 0.2
CV = 3
N_JOBS = -1
SCORING = 'r2'

EXPERIMENT_NAMES = [
    "regression_no_scaler",
    "regression_with_scaler",
    "regression_with_scaler_and_pca"
]

In [7]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def ensure_experiment(name):
    client = mlflow.tracking.MlflowClient()
    existing = client.get_experiment_by_name(name)
    if existing is not None:
        return existing.experiment_id
    else:
        return client.create_experiment(name)

def log_grid_results_to_csv(grid, path):
    df = pd.DataFrame(grid.cv_results_)
    df.to_csv(path, index=False)
    return path


In [8]:
def load_data(df):
    data = df
    X = data.drop(columns=['rent'])
    y = data.rent
    return X, y


In [9]:
def get_models_and_grids():
    models = {}

    models['LinearRegression'] = (
        LinearRegression(),
        {'model__fit_intercept': [True, False]}
    )

    models['Ridge'] = (
        Ridge(random_state=RANDOM_STATE),
        {'model__alpha': [0.01, 0.1, 1.0, 10.0]}
    )

    models['Lasso'] = (
        Lasso(random_state=RANDOM_STATE, max_iter=5000),
        {'model__alpha': [0.001, 0.01, 0.1, 1.0]}
    )

    models['ElasticNet'] = (
        ElasticNet(random_state=RANDOM_STATE, max_iter=5000),
        {'model__alpha': [0.001, 0.01, 0.1, 1.0], 'model__l1_ratio': [0.2, 0.5, 0.8]}
    )

    models['DecisionTree'] = (
        DecisionTreeRegressor(random_state=RANDOM_STATE),
        {'model__max_depth': [None, 5, 10], 'model__min_samples_split': [2, 5]}
    )

    models['RandomForest'] = (
        RandomForestRegressor(random_state=RANDOM_STATE),
        {'model__n_estimators': [50, 100], 'model__max_depth': [None, 10]}
    )

    models['GradientBoosting'] = (
        GradientBoostingRegressor(random_state=RANDOM_STATE),
        {'model__n_estimators': [50, 100], 'model__learning_rate': [0.05, 0.1]}
    )

    models['SVR'] = (
        SVR(),
        {'model__kernel': ['rbf', 'linear'], 'model__C': [0.1, 1.0, 10.0]}
    )

    models['KNeighbors'] = (
        KNeighborsRegressor(),
        {'model__n_neighbors': [3, 5, 9], 'model__weights': ['uniform', 'distance']}
    )

    # 🚀 XGBoost
    models['XGBoost'] = (
        XGBRegressor(objective='reg:squarederror', random_state=RANDOM_STATE, verbosity=0, n_jobs=-1),
        {
            'model__n_estimators': [50, 100],
            'model__max_depth': [3, 5, 7],
            'model__learning_rate': [0.05, 0.1],
            'model__subsample': [0.8, 1.0],
            'model__colsample_bytree': [0.8, 1.0]
        }
    )

    return models

In [10]:
def run_experiments(df):
    X, y = load_data(df)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

    # Detect categorical and numeric columns automatically
    cat_cols = X_train.select_dtypes(include=["object", "category"]).columns
    num_cols = X_train.select_dtypes(exclude=["object", "category"]).columns

    models_and_grids = get_models_and_grids()

    for exp_name in EXPERIMENT_NAMES:
        exp_id = ensure_experiment(exp_name)
        mlflow.set_experiment(exp_name)
        print(f"\n=== Running experiment: {exp_name} ===")

        # Define preprocessing per experiment
        if exp_name == "regression_no_scaler":
            numeric_transformer = "passthrough"
        elif exp_name == "regression_with_scaler":
            numeric_transformer = StandardScaler()
        else:  # with scaler + PCA
            numeric_transformer = Pipeline(
                [("scaler", StandardScaler()), ("pca", PCA())]
            )

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, num_cols),
                ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ]
        )

        for model_name, (estimator, grid) in models_and_grids.items():
            pipe = Pipeline(steps=[("preprocessor", preprocessor), ("model", estimator)])

            # If PCA present in pipeline, add grid for pca__n_components
            if exp_name == "regression_with_scaler_and_pca":
                grid = grid.copy()
                grid["preprocessor__num__pca__n_components"] = [0.95, 0.99, min(10, X_train.shape[1])]

            with mlflow.start_run(run_name=f"{model_name}_{datetime.now():%Y%m%d_%H%M%S}"):
                print(f"  ▶ Model: {model_name}")

                mlflow.log_param("experiment", exp_name)
                mlflow.log_param("model_name", model_name)

                gs = GridSearchCV(
                    pipe,
                    param_grid=grid,
                    cv=CV,
                    scoring=SCORING,
                    n_jobs=N_JOBS,
                    verbose=1,
                    return_train_score=False,
                )

                gs.fit(X_train, y_train)

                best = gs.best_estimator_
                y_pred = best.predict(X_test)
                r2 = r2_score(y_test, y_pred)
                test_rmse = rmse(y_test, y_pred)

                # Log results
                for k, v in gs.best_params_.items():
                    mlflow.log_param(k, v)
                mlflow.log_metric("r2_test", r2)
                mlflow.log_metric("rmse_test", test_rmse)

                # Log grid results and model
                tmpdir = tempfile.mkdtemp()
                grid_path = os.path.join(tmpdir, f"grid_results_{model_name}.csv")
                log_grid_results_to_csv(gs, grid_path)
                mlflow.log_artifact(grid_path, artifact_path="grid_search")

                mlflow.sklearn.log_model(best, artifact_path="model")

                print(f"    ✓ R2={r2:.4f}, RMSE={test_rmse:.4f}")

    print("\n🎯 All experiments complete! Launch MLflow UI with:")
    print("   mlflow ui --port 5000")

In [11]:
run_experiments(df)

C:\Users\Vinayak\AppData\Roaming\Python\Python311\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)



=== Running experiment: regression_no_scaler ===
  ▶ Model: LinearRegression
Fitting 3 folds for each of 2 candidates, totalling 6 fits


2025/12/26 14:55:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/26 14:56:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.7512, RMSE=51748.5503
  ▶ Model: Ridge
Fitting 3 folds for each of 4 candidates, totalling 12 fits


2025/12/26 14:56:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/26 14:56:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

    ✓ R2=0.5411, RMSE=70275.0440
  ▶ Model: KNeighbors
Fitting 3 folds for each of 18 candidates, totalling 54 fits


2025/12/26 15:58:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/26 15:58:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.7949, RMSE=46975.9123
  ▶ Model: XGBoost
Fitting 3 folds for each of 144 candidates, totalling 432 fits


C:\Users\Vinayak\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
2025/12/26 16:01:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/26 16:01:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ R2=0.7738, RMSE=49332.9897

🎯 All experiments complete! Launch MLflow UI with:
   mlflow ui --port 5000


In [14]:
# Random Forest with no scaler performs best at r2 score of 0.89, so i will be trying different params in detail
param_grid = {
        "n_estimators": [50, 100,500],
        "max_depth": [None, 20,50],
        "min_samples_split": [2, 5,10],
        "min_samples_leaf": [1, 2,5],
        "max_features": ["sqrt", "log2",None],
        "bootstrap": [True, False],
    }

In [15]:
def random_forest_detailed(df):
    # Split data
    X, y = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Identify categorical and numeric features
    cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_features = X.select_dtypes(exclude=['object', 'category']).columns.tolist()

    # Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
        ],
        remainder='passthrough'
    )

    # Model
    rf = RandomForestRegressor(random_state=42)

    # Pipeline
    pipeline = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('model', rf)
    ])

    # Define parameter grid
    param_grid = {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [5, 10, 15, None],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4],
        'model__max_features': ['sqrt', 'log2', None]
    }

    grid = list(ParameterGrid(param_grid))
    print(f"Total runs: {len(grid)}")

    # Loop through every combination and log separately
    for i, params in enumerate(grid):
        with mlflow.start_run(run_name=f"RF_Run_{i+1}"):

            # Set params
            pipeline.set_params(**params)
            pipeline.fit(X_train, y_train)

            # Predictions
            y_pred = pipeline.predict(X_test)

            # Metrics
            test_rmse = rmse(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("rmse", test_rmse)
            mlflow.log_metric("r2_score", r2)

            # Log model
            mlflow.sklearn.log_model(pipeline, name="model")

            print(f"✅ Run {i+1}/{len(grid)} logged | RMSE: {test_rmse:.4f} | R²: {r2:.4f}")




In [16]:
import time
import itertools
from sklearn.model_selection import ParameterGrid

In [17]:
random_forest_detailed(df)

Total runs: 324


2025/12/26 16:17:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 1/324 logged | RMSE: 53226.8306 | R²: 0.7367


2025/12/26 16:17:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 2/324 logged | RMSE: 53643.9916 | R²: 0.7326


...

✅ Run 322/324 logged | RMSE: 38162.0156 | R²: 0.8647


2025/12/26 17:57:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 323/324 logged | RMSE: 38096.8058 | R²: 0.8651


2025/12/26 17:58:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run 324/324 logged | RMSE: 38327.8200 | R²: 0.8635
